<a href="https://colab.research.google.com/github/gsbiel/cv-homography/blob/add-dlt/custom_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Computer Vision Assignment - 2**

Students: 
- Gabriel Silva Gaspar; 
- Sara Sampaio Gomes do Nascimento.

In [4]:
!pip install opencv-python==3.4.2.16
!pip install opencv-contrib-python==3.4.2.16

     |████████████████████████████████| 25.0MB 1.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
     |████████████████████████████████| 30.6MB 160kB/s 
  Found existing installation: opencv-contrib-python 4.1.2.30
    Uninstalling opencv-contrib-python-4.1.2.30:
      Successfully uninstalled opencv-contrib-python-4.1.2.30


In [138]:
import cv2 as cv
from matplotlib import pyplot as plt
import imutils
import numpy as np
import math

MIN_MATCH_COUNT = 10

In [144]:
# ----------------------------------------------------------------------------------------------------------

"""
  Essa função recebe duas imagens como argumento e retorna os pontos de 
  correspondência entre as duas.
"""

def get_correspondence_points(img1, img2):

  # Initiate SIFT detector
  sift = cv.xfeatures2d.SIFT_create()

  # find the keypoints and descriptors with SIFT
  kp1, des1 = sift.detectAndCompute(img1,None)
  kp2, des2 = sift.detectAndCompute(img2,None)
  # FLANN parameters
  # FLANN stands for Fast Library for Approximate Nearest Neighbors. 
  # It contains a collection of algorithms optimized for fast nearest neighbor 
  # search in large datasets and for high dimensional features. 
  # It works faster than BFMatcher for large datasets.
  # The variable index_params specifies the algorithm to be used, its related parameters etc. 
  # For algorithms like SIFT, SURF etc. you can pass following:
  FLANN_INDEX_KDTREE = 1
  index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
  # The variable search_params specifies the number of times the trees in the index should 
  # be recursively traversed. Higher values gives better precision, but also takes more time.
  #search_params = dict(checks=50)   # or pass empty dictionary
  search_params = dict()

  flann = cv.FlannBasedMatcher(index_params,search_params)
  matches = flann.knnMatch(des1,des2,k=2)

  # store all the good matches as per Lowe's ratio test.
  good = []
  for m,n in matches:
      if m.distance < 0.75*n.distance:
          good.append(m)

  if len(good)>MIN_MATCH_COUNT:
      src_pts = np.float32([ kp1[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
      dst_pts = np.float32([ kp2[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
      M, mask = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 5.0)
      matchesMask = mask.ravel().tolist()
  else:
    print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )

  return {
      "src_pts":src_pts,
      "dst_pts":dst_pts
  }

# ----------------------------------------------------------------------------------------------------------

"""
Essa função formata os pontos de correspondência retornados pela função FLANN 
para que fiquem como uma matriz de pontos em coordenadas homogêneas.
"""

def format_points(points):
  flatten_points = np.ravel(points)
  number_of_rows = int(len(flatten_points)/2)
  formatted_points = flatten_points.reshape(number_of_rows,2)
  ones = np.ones(formatted_points.shape[0])
  return np.c_[formatted_points,ones]

# ----------------------------------------------------------------------------------------------------------

"""
Função que teremos que implementar.
M e mask são os dados que ela deve retornar.
"""

def findHomography(src_pts, dst_pts):
  # Altera formato do numpy array
  src = format_points(src_pts)
  dst = format_points(dst_pts)

  # Obtém as matrizes de normalização dos pontos de cada imagem
  normalize_src = get_normalizing_matrix(src)
  normalize_dst = get_normalizing_matrix(dst)

  # Normaliza os pontos de cada imagem
  src = normalize_src.dot(src.transpose()).transpose()
  dst = normalize_dst.dot(dst.transpose()).transpose()

  A_list = []

  for index in range(0,src.shape[0]):
    # Para cada par de correspondências, cria a matriz A
    a11 = np.array([0, 0, 0])
    a12 = -dst[index][2]*src[index]
    a13 = -dst[index][1]*src[index]
    a21 = -a12
    a22 = np.copy(a11)
    a23 = -dst[index][0]*src[index]
    a1 = np.concatenate((a11, a12, a13), axis=None)
    a2 = np.concatenate((a21, a22, a23), axis=None)
    A = np.vstack((a1,a2))
    # Insere a matriz A em uma lista de matrizes
    A_list.append(A)

  # Converte a lista de matrizes em um numpy array
  A_stack = np.array(A_list)
  # Modela a lista de matrizes para se tornar uma pilha de matrizes
  A_stack = A_stack.reshape((A_stack.shape[0]*A_stack.shape[1], 9))
  
  # Decompõe a matriz A usando decomposição SVD
  u, s, vh = np.linalg.svd(A_stack, full_matrices=True)
  # Formata a solução do algoritmo DLT, ainda normalizada
  M_normalized = vh[:,-1].reshape((3,3))
  # Desnormaliza a solução do algoritmo DLT
  M = np.linalg.inv(normalize_src).dot(M_normalized.dot(normalize_dst))

  # Esse array indica se um ponto é um inlier ou outlier.
  # Ele deve ser construído a partir do RANSAC
  mask = np.array([
                   [0],
                   [1],
                   [1],
                   [0]
                   # ...
  ])
  return [M, mask]

# ----------------------------------------------------------------------------------------------------------

"""
Função que calcula a matriz, T, de normalização dos pontos de uma imagem
"""

def get_normalizing_matrix(points):
  # Calcula a soma dos elementos de cada coluna da matriz
  column_sums = points.sum(axis=0)
  # Cacula os valores médios de u(eixo x) e v(eixo y) da imagem
  u_mean_value = float(column_sums[0])/float(points.shape[0])
  v_mean_value = float(column_sums[1])/float(points.shape[0])
  # Calcula somatório do quadrado das distâncias de cada ponto da imagem com relação ao ponto médio
  sum_of_squares = 0
  n = points.shape[0]
  for index in range(0,n):
    u_diff = points[index][0] - u_mean_value
    v_diff = points[index][1] - v_mean_value
    sum_of_squares += math.sqrt((math.pow(u_diff,2) + math.pow(v_diff,2)))
  # Calcula o parâmetro s da matriz de normalização
  s = float(math.sqrt(2)*n)/float(sum_of_squares)
  # Calcula e retorna a matriz de normalização
  return s*np.array([ 
                     [1, 0, -u_mean_value],  
                     [0, 1, -v_mean_value],   
                     [0, 0,    float(1)/s]               
  ])

In [147]:
# Carrega imagens para testar a função de homografia
img1 = cv.imread('comicsStarWars01.jpg',0) # queryImage
img2 = cv.imread('comicsStarWars02.jpg',0) # trainImage

# Rotaciona a imagem para ficar melhor a visualização na hora de plotar
img1 = imutils.rotate_bound(img1,90)
img2 = imutils.rotate_bound(img2, 90)

# Obtém pontos de correspondência
correspondence_points = get_correspondence_points(img1, img2)

# Essa é a função que temos que desenvolver!
M, mask = findHomography(correspondence_points["src_pts"], correspondence_points["dst_pts"])


(3, 3)
[[ 1.33470580e-01 -8.67167227e-01 -4.08530160e+01]
 [ 3.97437679e-02 -3.47672656e-01 -7.66428106e+01]
 [ 1.26454670e-04 -3.41661611e-04 -5.51713410e-01]]
